In [21]:
#Importing lib's
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from statsmodels.multivariate.manova import MANOVA
from scipy import stats

In [22]:
#Loading the data
limnology_file = r"S:\Sem_4\Output\cleaned_limnology_data.xlsx"
enriched_file = r"S:\Sem_4\Output\enriched_lake_data.xlsx"

df_lim = pd.read_excel(limnology_file)
df_enriched = pd.read_excel(enriched_file)

In [23]:
#Cleaning Column Names
df_enriched.columns = [c.lower().replace(' ', '_').replace('(', '').replace(')', '').replace('/', '_') for c in df_enriched.columns]
df_lim.columns = [c.lower().replace(' ', '_').replace('(', '').replace(')', '').replace('/', '_') for c in df_lim.columns]
df_lim['lake_name'] = df_lim['lake_name'].astype(str).str.title().str.replace('_', ' ').str.strip()

In [24]:
#Seasonal Trend
#Preparing Date and Season variables
df_lim['date'] = pd.to_datetime(df_lim['date'])
df_lim['date_ordinal'] = df_lim['date'].apply(lambda x: x.toordinal())
df_lim['month'] = df_lim['date'].dt.month
df_lim['season'] = pd.cut(df_lim['month'], bins=[0, 3, 6, 9, 12], labels=['Winter', 'Spring', 'Summer', 'Fall'])

In [25]:
#Defining Trend Function
def run_seasonal_trend(group):
    #Defining the output structure for consistency
    output = {
        'n_obs': 0,
        'trend_slope_mgL_yr': np.nan,
        'p_value': np.nan,
        'r_squared': np.nan,
        'significant': False
    }

    #Checking for required columns
    req_cols = ['average_phosphorus_mg_l', 'date_ordinal', 'season']
    if not all(col in group.columns for col in req_cols):
        return pd.Series(output)

    #Filtering for valid data
    clean_group = group.dropna(subset=req_cols)
    output['n_obs'] = len(clean_group)

    #Checking for sufficient data (e.g., < 6 points is too few)
    if len(clean_group) < 6:
        return pd.Series(output)

    try:
        #Determine Formula
        #If we have multiple seasons, control for them. If not, just use Date.
        if clean_group['season'].nunique() > 1:
            formula = "average_phosphorus_mg_l ~ date_ordinal + C(season)"
        else:
            formula = "average_phosphorus_mg_l ~ date_ordinal"

        #Running Regression
        model = smf.ols(formula, data=clean_group).fit()

        #Extracting Results
        slope_per_day = model.params['date_ordinal']
        output['trend_slope_mgL_yr'] = slope_per_day * 365
        output['p_value'] = model.pvalues['date_ordinal']
        output['r_squared'] = model.rsquared
        output['significant'] = model.pvalues['date_ordinal'] < 0.05

    except Exception as e:
        #If regression fails (e.g. singular matrix), return the empty structure
        pass

    return pd.Series(output)

In [26]:
#Running the Analysis
cols_needed = ['date_ordinal', 'season', 'average_phosphorus_mg_l']
# Apply the function
trend_results = df_lim.groupby('lake_name')[cols_needed].apply(run_seasonal_trend).reset_index()
trend_results = trend_results.dropna(subset=['p_value']).sort_values('p_value')

In [27]:
#Filtering and Showing the Results
valid_results = trend_results.dropna(subset=['p_value']).sort_values('p_value')
print("Significant Trends:")
print(valid_results[valid_results['significant'] == True])

Significant Trends:
              lake_name  n_obs  trend_slope_mgL_yr   p_value  r_squared  \
36        Mountain Lake      6            0.000002  0.000081   0.776978   
24  Kashagawigamog Lake     23            0.000786  0.009806   0.402801   
45        Stocking Lake     10            0.001059  0.027134   0.747107   
19            Gull Lake     10            0.002823  0.038410   0.625009   

    significant  
36         True  
24         True  
45         True  
19         True  


In [28]:
#ANCOVA (Phosphorus ~ Max Depth + Capacity Status)
#Comparing Capacity Status controlling for Depth
print("\nANCOVA Results")
ancova_data = df_enriched.dropna(subset=['average_phosphorus_mg_l', 'max_depth_m', 'capacity_status_2016'])
ancova_data = ancova_data[ancova_data['capacity_status_2016'] != 'Not at Capacity/At Capacity']

model_ancova = smf.ols("average_phosphorus_mg_l ~ max_depth_m + C(capacity_status_2016)", data=ancova_data).fit()
print(model_ancova.summary())

#Extracting Results table
ancova_summary = pd.DataFrame({
    'Coefficient': model_ancova.params,
    'Std Error': model_ancova.bse,
    'T-Value': model_ancova.tvalues,
    'P-Value': model_ancova.pvalues
})
#Adding model fit statistics
ancova_stats = pd.DataFrame({
    'Metric': ['R-Squared', 'Adj. R-Squared', 'F-Statistic', 'Prob (F-Stat)'],
    'Value': [model_ancova.rsquared, model_ancova.rsquared_adj, model_ancova.fvalue, model_ancova.f_pvalue]
})


ANCOVA Results
                               OLS Regression Results                              
Dep. Variable:     average_phosphorus_mg_l   R-squared:                       0.300
Model:                                 OLS   Adj. R-squared:                  0.246
Method:                      Least Squares   F-statistic:                     5.566
Date:                     Sat, 06 Dec 2025   Prob (F-statistic):            0.00972
Time:                             16:21:20   Log-Likelihood:                 157.10
No. Observations:                       29   AIC:                            -308.2
Df Residuals:                           26   BIC:                            -304.1
Df Model:                                2                                         
Covariance Type:                 nonrobust                                         
                                                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------

In [29]:
#MANOVA (Water Quality Profile ~ Capacity Status)
print("\nMANOVA Results")
manova_cols = ['average_phosphorus_mg_l', 'conductivity_us_cm', 'ph_meter']
manova_data = df_enriched.dropna(subset=manova_cols + ['capacity_status_2016'])
manova_data = manova_data[manova_data['capacity_status_2016'] != 'Not at Capacity/At Capacity']

ma = MANOVA.from_formula('average_phosphorus_mg_l + conductivity_us_cm + ph_meter ~ C(capacity_status_2016)', data=manova_data)
manova_results = ma.mv_test()
print(ma.mv_test())

#Extracting the specific table for our term of interest
manova_table = manova_results.results['C(capacity_status_2016)']['stat']


MANOVA Results
                   Multivariate linear model
                                                                
----------------------------------------------------------------
       Intercept         Value   Num DF  Den DF  F Value  Pr > F
----------------------------------------------------------------
          Wilks' lambda   0.0023 3.0000 25.0000 3680.2021 0.0000
         Pillai's trace   0.9977 3.0000 25.0000 3680.2021 0.0000
 Hotelling-Lawley trace 441.6243 3.0000 25.0000 3680.2021 0.0000
    Roy's greatest root 441.6243 3.0000 25.0000 3680.2021 0.0000
----------------------------------------------------------------
                                                                
----------------------------------------------------------------
    C(capacity_status_2016) Value  Num DF  Den DF F Value Pr > F
----------------------------------------------------------------
              Wilks' lambda 0.8033 3.0000 25.0000  2.0408 0.1338
             Pillai's trace 0

In [30]:
#Saving to Excel
output_path = r"S:\Sem_4\Output\statistical_results_summary.xlsx"

with pd.ExcelWriter(output_path) as writer:
    trend_results.to_excel(writer, sheet_name='1. Seasonal Trends', index=False)
    ancova_summary.to_excel(writer, sheet_name='2. ANCOVA Coefficients')
    ancova_stats.to_excel(writer, sheet_name='2. ANCOVA Stats', index=False)
    manova_table.to_excel(writer, sheet_name='3. MANOVA Results')